<a href="https://colab.research.google.com/github/bryanz994/neo4j-assignment/blob/main/Neo4j_Workshop_KG_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neo4j Workshop - Knowledge Graph Game

We will use a Neo4j graph database created on the [Neo4j sandbox](https://neo4j.com/sandbox/).

After creating a Neo4j sandbox instance, you can access the connection details by clicking on the top right down arrow and picking the *Connection details* tab.  

You will need 2 things:
* Password  
* Bolt URL   

<img src="sandbox_details.png" alt="Sandbox Details" width="100%" title="Sandbox Details">  

In [ ]:
# Install Neo4j GDS Python Client
import sys
!{sys.executable} -m pip install graphdatascience

# Import our GDS entry point
from graphdatascience import GraphDataScience

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.7/938.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.11.0-py3-none-any.whl size=259793 sha256=fdf800689698b24fcd6bf5f2ebfb70be2b8bdb7eb038b19712d93945865fa39c
  Stored in directory: /root/.cache/pip/wheels/5f/9b/72/d893916a92e5bd362c3c28dc8dacc060419009e5a8cebb9461
Successfully built neo4j


### Instantiate your GDS Session

Use Neo4j/Bolt URI and credentials according to your setup  

For local standalone instance Bolt connection without auth    
`gds = GraphDataScience("bolt://localhost:7687", auth=None)`  

For local standalone instance Bolt connection with auth    
`gds = GraphDataScience("bolt://localhost:7687", auth=("neo4j", "<password>"))`  

For remote cluster Neo4j connection with auth  
`gds = GraphDataScience("neo4j://<FQDN or IP Address>:7687", auth=("neo4j", "<password>"))`  

For remote standalone instance Bolt connection with auth   
`gds = GraphDataScience("bolt://<FQDN or IP Address>:7687", auth=("neo4j", "<password>"))`

In [ ]:
# >> Update the password and the URL here <<
gds = GraphDataScience("bolt://<your-ip-here>:7687", auth=("neo4j", "<password>"))

### Cleaning the database or making it ready for a rerun of the notebook.
We are starting with a fresh clean database, however if the database was previously loaded, we have the option to clear it out first here. Then we will use it to load the data from CSV files, running [Cypher](https://neo4j.com/developer/cypher/) queries. The RELOAD_DATA flag can be used to skip this step for experimenting with different algorithms later on.

In [ ]:
# Set flag to control reloading of all data
RELOAD_DATA = True


if RELOAD_DATA: # Delete all, takes few miniutes on a full database
  gds.run_cypher(
      """
      MATCH (n) CALL {
        WITH n
        DETACH DELETE n
      } IN TRANSACTIONS OF 100 ROWS;
      """
  )
else: # Run a match statement
  gds.run_cypher(
      """
      MATCH (p:Person) RETURN p;
      """
  )

### Test reading some data

Using [LOAD CSV](https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/), we are loading csv files into the database, creating the graph on the fly.  
The first cell is to test the file access, by reading it and showing only the first 5 rows.  

In [ ]:
# Checking if we can access the education dataset
educationListCSV=gds.run_cypher(
"""
LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_education.csv' AS row
RETURN row.name, row.passportnumber, row.nameofinstitution, row.course, row.country, row.startyear, row.endyear limit 5;
"""
)

# The object returned is a Pandas Data Frame, so we can explore using standard Pandas methods
educationListCSV.head(5)

,row.name,row.passportnumber,row.nameofinstitution,row.course,row.country,row.startyear,row.endyear
0,Anil Kumar,MNO9101PQR,Raika Primary School,-,Malaysia,1978,1983
1,Anil Kumar,MNO9101PQR,Baranagar Middle School,All India Secondary School Examination,Malaysia,1984,1985
2,Anil Kumar,MNO9101PQR,St George’s Junior High School,-,Malaysia,1986,1987
3,Anil Kumar,MNO9101PQR,St George’s Senior High School,All India Senior School Certificate Examination,Malaysia,1988,1989
4,Anil Kumar,MNO9101PQR,Smart National University of Vietnam,Bachelor of Business Administration,Vietnam,1992,1995


In [ ]:
# Checking if we can access the employment dataset
employmentListCSV=gds.run_cypher(
"""
LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_work.csv' AS row
RETURN row.passportnumber, row.designation, row.nameoforganization limit 5;
"""
)

# The object returned is a Pandas Data Frame, so we can explore using standard Pandas methods
employmentListCSV.head(5)

,row.passportnumber,row.designation,row.nameoforganization
0,ABC1234DEF,Vice President Global Marketing,Moon Microsystems
1,ABC1234DEF,Vice President Regional Marketing,PA Technologies
2,ABC1234DEF,Senior Marketing Director,Ultimate Pte Ltd
3,GHI5678JKL,Founding Editor in Chief,Journal of Engineering Science and Technology
4,GHI5678JKL,Director and Chairman of the Higher Education ...,British Malaysian Chamber of Commerce


In [ ]:
# Checking if we can access the travel dataset
travelListCSV=gds.run_cypher(
"""
LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_trips.csv' AS row
RETURN row.passportnumber, row.citizenship, row.arrivalcountry, row.departuredate, row.arrivaldate  limit 5;
"""
)

# The object returned is a Pandas Data Frame, so we can explore using standard Pandas methods
travelListCSV.head(5)

,row.passportnumber,row.citizenship,row.arrivalcountry,row.departuredate,row.arrivaldate
0,ABC1234DEF,Singapore,Hawaii,"Thursday, 12 April 2018","Thursday, 12 April 2018"
1,ABC1234DEF,Singapore,Indonesia,"Sunday, 2 June 2019","Sunday, 2 June 2019"
2,ABC1234DEF,Singapore,Korea,"Friday, 12 March 2021","Friday, 12 March 2021"
3,ABC1234DEF,Singapore,Vietnam,"Sunday, 26 September 2021","Sunday, 26 September 2021"
4,ABC1234DEF,Singapore,Dubai,"Saturday, 11 December 2021","Saturday, 11 December 2021"


In [ ]:
# Checking if we can access the transactions data set
transactionListCSV=gds.run_cypher(
"""
LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_transaction.csv' AS row
RETURN row.passportnumber, row.cardnumber, row.country, row.merchant, row.amount limit 5;
"""
)

# The object returned is a Pandas Data Frame, so we can explore using standard Pandas methods
transactionListCSV.head(5)

,row.passportnumber,row.cardnumber,row.country,row.merchant,row.amount
0,ABC1234DEF,2345-6789-1011 -1213-1415,Singapore,"5 SENSES BISTRO, SINGAPORE",$40.35
1,ABC1234DEF,2345-6789-1011 -1213-1415,Singapore,"A.LAB, SINGAPORE",$250.22
2,ABC1234DEF,2345-6789-1011 -1213-1415,Singapore,"B*DAZZLE, SINGAPORE",$22.50
3,ABC1234DEF,2345-6789-1011 -1213-1415,Singapore,"CAFE KURIKO, SINGAPORE",$12.95
4,ABC1234DEF,2345-6789-1011 -1213-1415,Singapore,"DDM FASHION, SINGAPORE",$55.85


### Creating constraints and indexes

For data integrity, we will create [constraints](https://neo4j.com/docs/cypher-manual/current/constraints/) to have a robust graph data model. Each constraint enforces uniqueness of an identifier for a given label. An index is also created for the name property on Client nodes, this allows fast lookups when querying clients by name.

In [ ]:
# Create index and constraint
CONSTRAINTS = [
    "CREATE CONSTRAINT personPassportNumberConstraint IF NOT EXISTS FOR (p:Person) REQUIRE p.passportnumber IS UNIQUE;",
    "CREATE CONSTRAINT countryNameConstraint IF NOT EXISTS FOR (c:Country) REQUIRE c.name IS UNIQUE;",
    "CREATE CONSTRAINT institutionNameConstraint IF NOT EXISTS FOR (i:Institution) REQUIRE i.name IS UNIQUE;",
    "CREATE CONSTRAINT merchantNameConstraint IF NOT EXISTS FOR (m:Merchant) REQUIRE m.name IS UNIQUE;",
    "CREATE INDEX organizationNameConstraint IF NOT EXISTS FOR (o:Organization) ON o.organization;"
]

for c in CONSTRAINTS:
    gds.run_cypher(c)

In [ ]:
# Verify constraint created
gds.run_cypher("SHOW CONSTRAINTS")

,id,name,type,entityType,labelsOrTypes,properties,ownedIndex,propertyType
0,14,BankConstraint,UNIQUENESS,NODE,[Bank],[id],BankConstraint,None
1,18,CashInConstraint,UNIQUENESS,NODE,[CashIn],[globalStep],CashInConstraint,None
2,20,CashOutConstraint,UNIQUENESS,NODE,[CashOut],[globalStep],CashOutConstraint,None
3,4,ClientConstraint,UNIQUENESS,NODE,[Client],[id],ClientConstraint,None
4,6,EmailConstraint,UNIQUENESS,NODE,[Email],[email],EmailConstraint,None
5,12,MerchantConstraint,UNIQUENESS,NODE,[Merchant],[id],MerchantConstraint,None
6,24,PaymentConstraint,UNIQUENESS,NODE,[Payment],[globalStep],PaymentConstraint,None
7,8,PhoneConstraint,UNIQUENESS,NODE,[Phone],[phoneNumber],PhoneConstraint,None
8,10,SSNConstraint,UNIQUENESS,NODE,[SSN],[ssn],SSNConstraint,None
9,16,TransactionConstraint,UNIQUENESS,NODE,[Transaction],[globalStep],TransactionConstraint,None


In [ ]:
# Verify index created
gds.run_cypher("SHOW INDEX")

,id,name,state,populationPercent,type,entityType,labelsOrTypes,properties,indexProvider,owningConstraint,lastRead,readCount
0,13,BankConstraint,ONLINE,100.0,RANGE,NODE,[Bank],[id],range-1.0,BankConstraint,2023-08-24T07:00:27.490000000+00:00,2246
1,17,CashInConstraint,ONLINE,100.0,RANGE,NODE,[CashIn],[globalStep],range-1.0,CashInConstraint,2023-08-24T07:02:21.197000000+00:00,211659
2,19,CashOutConstraint,ONLINE,100.0,RANGE,NODE,[CashOut],[globalStep],range-1.0,CashOutConstraint,2023-08-24T07:02:58.201000000+00:00,112407
3,3,ClientConstraint,ONLINE,100.0,RANGE,NODE,[Client],[id],range-1.0,ClientConstraint,2023-08-24T07:45:46.395000000+00:00,170063
4,26,ClientFraudIndex,ONLINE,100.0,RANGE,NODE,[Client],[fraud_group],range-1.0,None,2023-08-24T07:46:22.896000000+00:00,29
5,25,ClientNameIndex,ONLINE,100.0,RANGE,NODE,[Client],[name],range-1.0,None,2023-08-24T07:28:52.871000000+00:00,1
6,5,EmailConstraint,ONLINE,100.0,RANGE,NODE,[Email],[email],range-1.0,EmailConstraint,2023-08-24T07:00:18.985000000+00:00,3289
7,11,MerchantConstraint,ONLINE,100.0,RANGE,NODE,[Merchant],[id],range-1.0,MerchantConstraint,2023-08-24T07:03:40.583000000+00:00,144218
8,23,PaymentConstraint,ONLINE,100.0,RANGE,NODE,[Payment],[globalStep],range-1.0,PaymentConstraint,2023-08-24T07:03:40.583000000+00:00,107238
9,7,PhoneConstraint,ONLINE,100.0,RANGE,NODE,[Phone],[phoneNumber],range-1.0,PhoneConstraint,2023-08-24T07:00:18.884000000+00:00,3291


### Loading all the data

We will load 4 csv files:  
* one for education   
* one for employment  
* one for travel
* one for transaction

We can see how each node is created with a label and at least one property.  
We see all the relationships between all the nodes, we represent each transaction as a relationship between its participants.  

In [ ]:
# Load Education data
gds.run_cypher(
"""
    LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_education.csv' AS row
    MERGE (person:Person {passportnumber: row.passportnumber})
      ON CREATE SET
        person.name = row.name
    MERGE (institution:Institution {name: row.nameofinstitution})
    MERGE (country:Country {name: row.country})
    CREATE (course:Course {name: row.course})
    MERGE (person)-[:STUDIED_AT]->(institution)
    MERGE (institution)<-[:FROM]-(course)
    MERGE (country)<-[:FROM]-(institution)
    MERGE (course)<-[:MATRICULATED {year:row.startyear}]-(person)-[:GRADUATED {year:row.endyear}]->(course);
"""
)

# Load Employment data
gds.run_cypher(
"""
  LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_work.csv' AS row
  MERGE (organization:Organization {designation: row.designation, organization: row.nameoforganization, startyear: row.startyear, endyear: row.endyear})
  MERGE (country:Country {name: row.country})
  WITH organization, country, row
  MATCH (person:Person {passportnumber: row.passportnumber})
  MERGE (person)-[:EMPLOYED]->(organization);
"""
)

# Load Travel data
gds.run_cypher(
"""
  LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_trips.csv' AS row
  MERGE (citizenship:Country {name: row.citizenship})
  MERGE (arrival:Country {name: row.arrivalcountry})
  WITH  citizenship, arrival, row
  MATCH (person:Person {passportnumber: row.passportnumber})
  MERGE (person)-[:TRAVELLED {departuredate: row.departuredate, arrivaldate: row.arrivaldate}]->(arrival)
  MERGE (person)-[:CITIZEN_OF]->(citizenship);
"""
)

# Load Transaction Data
gds.run_cypher(
"""
  LOAD CSV WITH HEADERS FROM 'https://gist.githubusercontent.com/maruthiprithivi/10b456c74ba99a35a52caaffafb9d3dc/raw/a46af9c6c4bf875ded877140c112e9ff36f8f2e8/sng_transaction.csv' AS row
  MERGE (card:Card {cardnumber: row.cardnumber})
  MERGE (country:Country {name: row.country})
  MERGE (merchant:Merchant {name: row.merchant})
  WITH card, country, merchant, row
  MATCH (person:Person {passportnumber: row.passportnumber})
  MERGE (person)-[:HAS_CARD]->(card)
  MERGE (merchant)<-[:TRANSACTED {amount: row.amount, transactiondate: row.transactiondate}]-(card)
  MERGE (transaction)-[:FROM]->(merchant);
"""
)


We have now taken a series of flat data sources and constructed a rich graph representation of the connections present in the sample dataset. At this point we have the following data model :


<img src="graph_schema.png" alt="Initial graph data model" width="75%"  title="Initial Graph Data Model">  


### Graph Exploration with Cypher using Neo4j Web Browser
P.S: ***Switch to Neo4j Web Browser***

Copy and run the queries below in Neo4j Web Browser

---

#### **Basic information retrieval from the Graph**

---

#### Retrieve Nodes/Vertex of a specific "Label"

> Question: Retrieve all Person Node

> Solution: `MATCH (p:Person) RETURN p;`


#### Retrieve specific properties of a Node/Vertex

> Question: Retrieve name and passport number of Person Nodes

> Solution: `MATCH (p:Person) RETURN p.name as Name, p.passportnumber as PassportNumber;`


#### Retrieve a specific Node/Vertex using a specific property

> Question: Retrieve a Person Node with a specific passport number of GHI5678JKL

> Solution: `MATCH (p:Person {passportnumber: "GHI5678JKL"}) RETURN p;`


#### Rerieve Relationship/Edge of a specific "Type"
> Question: Retrieve all "EMPLOYED" Relationship/Edge | Since Relationships/Edges can only exist between two Nodes/Vertices, we will also retrieve the Nodes/Vertices that are connected to the Relationship/Edge

> Solution: `MATCH (p)-[r:EMPLOYED]->(o) RETURN p, r, o; `

#### Retrieve specific properties of a Relationship/Edge
> Question: Retrieve the arrival and departure dates of the "TRAVELLED" Relationship/Edge

> `MATCH ()-[r:TRAVELLED]->() RETURN r.arrivaldate as ArrivalDate, r.departuredate as DepartureDate;`


---

#### **Graph Traversals**

---

#### Traverse the Graph using a specific Node/Vertex Label as the starting point and retrieve all the Nodes/Vertices that are connected to it by One Degree away.
> Question: Retreive all the relationships and nodes that are connected to "Institution" Nodes by One Degree away in either directions

> Solution: `MATCH (i:Institution)-[r]-(n) RETURN i, r, n;`

*Note*: You might have noticed that there are a lot more Relationships/Vertices that are connecting the First degree Nodes/Vertices of the "Institution" Nodes.
When you are doing Graph Traversals, you can also specify the direction and the path of the Traversal.


#### Perform a direction specific traversal from the "Institution" Nodes
> Question: Retreive all the OUTGOING relationships and nodes that are connected to "Institution" Nodes

> Solution: `MATCH (i:Institution)-[r]->(n) RETURN i, r, n;`

#### Perform a Relationship/Vertex type specific traversal from the "Institution" Nodes
> Question: Retreive all the nodes that are connected to "Institution" Nodes via the STUDENT_AT relationship

> Solution: `MATCH (i:Institution)-[r:STUDIED_AT]-(n) RETURN i, r, n;`

### Graph Exploration with Cypher using Collab Notebook

In [ ]:
# @title Find out the countries that each person travelled to
# Retrieve Nodes/Vertex of a specific "Label"
gds.run_cypher(
"""
MATCH (p:Person)-[:TRAVELLED]->(c:Country)
RETURN p.name as `Person Name`, c.name as `Country Name`
"""
)

In [ ]:
# @title Find out if there are any common travel destination between the Person nodes
gds.run_cypher(
"""
MATCH (p:Person)-[t:TRAVELLED]->(c:Country)
RETURN c.name as `Country Name`, t.arrivaldate as `Arrival Date` ,COLLECT(p.name) as `Person Name`
ORDER BY `Arrival Date`
"""
)

In [ ]:
# @title Find out the transaction history at specific Merchants for each Person node { display-mode: "form" }
gds.run_cypher(
"""
MATCH (p:Person)-[:HAS_CARD]->(:Card)-[:TRANSACTED]->(m:Merchant)
RETURN p.name as `Person Name`, m.name as `Merchant Name`
"""
)


In [ ]:
# @title Find out if there are any common point of transaction between the Person nodes { display-mode: "form" }
gds.run_cypher(
"""
MATCH (p:Person)-[:HAS_CARD]->(:Card)-[t:TRANSACTED]->(m:Merchant)
RETURN m.name as `Merchant Name`, t.transactiondate as `Transaction Date`, COLLECT(p.name) as `Person Name`
"""
)

#### **Graph Projection using Graph Data Science**

Using Eigenvector centrality algorithm to find out the most influential institution

Note: *High eigenvector score means that a node is connected to many nodes who themselves have high scores*



In [54]:
# My first graph project name to use eigenvenctor centrality algorithm
graphName = 'influentialInstitution'

# Remove existing projection with the same name, in case of a re run of the notebook
if gds.graph.exists(graphName).exists:
    gds.graph.drop(gds.graph.get(graphName))

graph, project_result = gds.graph.project(
  'influentialInstitution', # Nodes to be added in the projection
  ["Person","Institution"], # Relationships to be added in the projection
  ["STUDIED_AT"]
)

In [46]:
# This means that Smart National University of Vietnam is highly centralized and connected to other nodes with high score.
gds.run_cypher(
"""
CALL gds.eigenvector.stream('influentialInstitution')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score
ORDER BY score DESC, name ASC LIMIT 10
"""
)

,name,score
0,Smart National University of Vietnam,0.696676
1,Baranagar Middle School,0.239056
2,Bukit Panjang Primary School,0.239056
3,Maju Secondary School,0.239056
4,Raika Primary School,0.239056
5,Sekolah Menengah Kebangsaan Chung Hua,0.239056
6,Sekolah Rendah Kebangsaan Peringgit,0.239056
7,St George’s Junior High School,0.239056
8,St George’s Senior High School,0.239056
9,Temasek Junior College,0.239056


In [ ]:
# @title Find out which Person node studied in Smart National University of Vietnam { display-mode: "form" }
gds.run_cypher(
"""
MATCH (p:Person)-[:STUDIED_AT]->(i:Institution { name: "Smart National University of Vietnam"})
RETURN p.name
"""
)